In [7]:
import ngsolve
from netgen.geom2d import unit_square
import netgen.gui
from netgen.meshing import *
from ngsolve.webgui import Draw
import numpy as np

%gui tk

In [8]:
# Defining constants

epsilon = 0.02
gamma = 50
M = 2

h_max = 0.1
tau = 0.1

In [9]:
# Setting up the mesh

ngmesh = Mesh(dim=2)

N= int(1/h_max)

"""
Code for the square mesh taken from the documentation page
https://docu.ngsolve.org/latest/i-tutorials/unit-4.3-manualmesh/manualmeshing.html
"""


pnums = []
for i in range(N + 1):
    for j in range(N + 1):
        pnums.append(ngmesh.Add(MeshPoint(Pnt(i / N, j / N, 0))))

idx_dom = ngmesh.AddRegion("mat", dim=2)
for j in range(N):
    for i in range(N):
        ngmesh.Add(Element2D(idx_dom, [pnums[i + j * (N + 1)],
                               pnums[i + (j + 1) * (N + 1)],
                               pnums[i + 1 + (j + 1) * (N + 1)],
                               pnums[i + 1 + j * (N + 1)]]))
        
# horizontal boundaries
for i in range(N):
   ngmesh.Add(Element1D([pnums[N + i * (N + 1)],
                       pnums[N + (i + 1) * (N + 1)]], index=1))
   ngmesh.Add(Element1D([pnums[0 + i * (N + 1)], pnums[0 + (i + 1) * (N + 1)]], index=1))

# vertical boundaries
for i in range(N):
   ngmesh.Add(Element1D([pnums[i], pnums[i + 1]], index=2))
   ngmesh.Add(Element1D([pnums[i + N * (N + 1)], pnums[i + 1 + N * (N + 1)]], index=2))


In [ ]:
mesh = ngsolve.Mesh(ngmesh)
Draw(mesh)

In [44]:
# Defining finite element space

V_phi = ngsolve.H1(mesh,order=1)
V_mu  = ngsolve.H1(mesh,order=1)

phi_n, v = V_phi.TnT()
mu_n, w = V_mu.TnT()

gfu = ngsolve.GridFunction(V_phi)



In [54]:
# Initialisation of phi_0

phi_0 = np.random.uniform(low = -1, high = 1, size = mesh.nv)
rand_expr = ngsolve.GridFunction(V_phi)
for i in range(mesh.nv):
    rand_expr.vec[i] = phi_0[i]

gfu.Set(rand_expr) 


In [ ]:
Draw(gfu)